
Let's experiment with a bunch more MCP Servers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [ ]:
import os
from pathlib import Path
from agents.mcp import MCPServerStdio

# Base environment with Node.js in PATH
def get_mcp_env(**extra_vars):
    env = os.environ.copy()
    env["PATH"] = f"/home/dneelapu/.nvm/versions/node/v22.21.0/bin:{env.get('PATH', '')}"
    env.update(extra_vars)
    return env

# For LibSQL Memory
Path("./memory").mkdir(exist_ok=True)
params = {
    "command": "npx",
    "args": ["mcp-memory-libsql"],
    "env": get_mcp_env(LIBSQL_URL="file:./memory/dhanu.db")
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

In [11]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Dhanu. I'm an AI Product engineer. I'm learning a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [12]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Dhanu! It's great that you're learning about AI Agents and the MCP protocol. The MCP protocol sounds like a powerful way to connect AI agents with various tools and resources, making integrations smoother and more capable. If you have any questions or want to discuss anything specific about AI agents or MCP, feel free to ask!

In [14]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's dhanu. What do you know about me?")
    display(Markdown(result.final_output))

You are Dhanu, an AI Product engineer. You are currently learning a course about AI Agents including the MCP protocol. Is there anything specific you would like me to remember or help you with regarding this?

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [15]:
import os
key = os.getenv("BRAVE_API_KEY")
print(f"Key exists: {key is not None}")
print(f"Key length: {len(key) if key else 0}")

Key exists: True
Key length: 31


In [17]:
import os
from agents.mcp import MCPServerStdio

# For Brave Search
params = {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": get_mcp_env(BRAVE_API_KEY=os.getenv("BRAVE_API_KEY"))
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

print(mcp_tools)

[Tool(name='brave_web_search', title=None, description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, outputSchema=None, icons=None, annotations=None, meta=None), Tool(name='brave_local_search', title=None, description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, res

In [18]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [19]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Here's a summary of the latest news and outlook for Amazon (AMZN) stock as of October 28, 2025:

1. **Current Price Outlook**: The average stock price target from analysts is approximately **$266.26**, indicating about a **17.16% potential increase** from its current level.

2. **Analyst Ratings**: Amazon stock has received a consensus rating of **"Strong Buy"** from 46 analysts, showing strong confidence in its performance over the next year. 

3. **Future Predictions**: Some forecasts predict a potential price of **$267.88** within the next year, reflecting positive sentiment towards the stock driven by favorable market conditions and strong business fundamentals.

4. **Market Sentiment**: There is optimism regarding the stock's ability to break out further, as suggested by some analysts citing strong growth prospects.

In summary, the outlook for Amazon stock appears positive, with predictions of continued upward momentum through late 2025 and into 2026.

### As usual, check out the trace:

https://platform.openai.com/traces

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# And back to the 2nd type: the Polygon.io MCP Server

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [20]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [21]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=268.81, high=269.12, low=264.6501, open=264.88, timestamp=1761595200000, volume=44888052.0, vwap=267.0053)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [22]:
from market import get_share_price
get_share_price("AAPL")

268.81

In [23]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

268.81

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [24]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', title=None, description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [25]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $268.81.